In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import urllib.request
import matplotlib.pyplot as plt
from scipy.stats import zscore

In [2]:
list_metricas = ['average_clustering', 'coefficient_distribution_degree', 'density', 'average_degree', 'average_short_path']#, 'number_of_nodes']


list_variables_a = [
    #'Patrimonio_Liquido',
    #'Cotas_Emitidas',
    #'Valor_Patrimonial_Cotas',
    'Percentual_Rentabilidade_Efetiva_Mes',
    'Percentual_Rentabilidade_Patrimonial_Mes',
    #'Percentual_Dividend_Yield_Mes',
    #'Percentual_Amortizacao_Cotas_Mes',
]


list_variables_b = [
    'Valor_Ativo',
    'Percentual_Despesas_Taxa_Administracao',
    'Percentual_Despesas_Agente_Custodiante',
    'Total_Necessidades_Liquidez',
    'Disponibilidades',
    'Titulos_Publicos',
    'Titulos_Privados',
    'Fundos_Renda_Fixa',
    'Total_Investido',
    'Direitos_Bens_Imoveis',
    'Terrenos',
    'Imoveis_Renda_Acabados',
    'Imoveis_Renda_Construcao',
    'Imoveis_Venda_Acabados',
    'Imoveis_Venda_Construcao',
    'Outros_Direitos_Reais',
    'Acoes',
    'Debentures',
    'Bonus_Subscricao',
    'Certificados_Deposito_Valores_Mobiliarios',
    'Cedulas_Debentures',
    'Fundo_Acoes',
    'FIP',
    'FII',
    'FDIC',
    'Outras_Cotas_FI',
    'Notas_Promissorias',
    'Acoes_Sociedades_Atividades_FII',
    'Cotas_Sociedades_Atividades_FII',
    'CEPAC',
    'CRI',
    'Letras_Hipotecarias',
    'LCI',
    'LIG',
    'Outros_Valores_Mobliarios',
    'Valores_Receber',
    'Contas_Receber_Aluguel',
    'Contas_Receber_Venda_Imoveis',
    'Outros_Valores_Receber',
    'Rendimentos_Distribuir',
    'Taxa_Administracao_Pagar',
    'Taxa_Performance_Pagar',
    'Obrigacoes_Aquisicao_Imoveis',
    'Adiantamento_Venda_Imoveis',
    'Adiantamento_Alugueis',
    'Obrigacoes_Securitizacao_Recebiveis',
    'Instrumentos_Financeiros_Derivativos',
    'Provisoes_Contigencias',
    'Outros_Valores_Pagar',
    'Total_Passivo',
 ]

list_variables =  list_variables_b #list_variables_a +

list_cat = [
    'Mandato',
    'Segmento_Atuacao',
    'Publico_Alvo',
    'Tipo_Gestao',
]

ds_fii = xr.open_dataset('dataset/cvm/dataset_cvm_metrica_mensal.nc').fillna(0)
df_dados_fii = pd.read_csv('dataset/cvm/dataset_cvm_metrica_mensal_dados.csv', index_col='CNPJ_Fundo')

df_fii = ds_fii.to_array().stack(var=['variable', 'metrica']).to_pandas()

df_fii[[ (v, 'coefficient_distribution_degree') for v in list_variables ]] = df_fii[[ (v, 'coefficient_distribution_degree') for v in list_variables ]][df_fii[[ (v, 'coefficient_distribution_degree') for v in list_variables ]].apply(zscore).abs() <= 1].fillna(0)
df_fii_cat =  df_dados_fii[list_cat]

df_fii = df_fii[ [ (v, m) for v in list_variables for m in list_metricas ] ]
df_fii[['TIR']] = df_dados_fii[['TIR']][df_dados_fii[['TIR']].fillna(0).apply(zscore).abs() <= 1].fillna(0)

cnpj_meses = xr.open_dataset('dataset/cvm/dataset_cvm_metrica_mensal.nc').fillna(0).Acoes.sel(metrica='number_of_nodes').to_pandas()

In [3]:
import copy

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import median_absolute_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import OneHotEncoder

In [4]:
def dataset_filter(df, tamanho_series):
    list_cnpj = cnpj_meses[cnpj_meses >= tamanho_series].index.values
    df_out = df[df.index.isin(list_cnpj)]
    
    return df_out

In [5]:
def select_features_rfr(df_fii, tamanho_series, num_test=10):
    df_fii_filter = dataset_filter(df_fii, tamanho_series)
    
    df_features = df_fii_filter.drop(columns=[('TIR', '')])
    df_target = df_fii_filter[('TIR', '')]
    
    prep = OneHotEncoder()
    prep.fit(df_dados_fii[df_dados_fii.index.isin(df_fii_filter.index.values)][list_cat])
    
    preprocessing = prep.transform(df_dados_fii[df_dados_fii.index.isin(df_fii_filter.index.values)][list_cat]).toarray()
    
    
    prep_cols = []
    for i in prep.categories_:
        prep_cols += list(i)
    
    prep_cols_tuple = []
    for i in prep_cols:
        prep_cols_tuple.append((i, ''))
        
    mi = pd.MultiIndex.from_tuples(list(df_features.columns.values) + prep_cols_tuple, names=('variable', 'metric'))
        
    df_features_values = pd.DataFrame(columns=mi).T
    mae_erros = []
    medae_erros = []
    
    for i in range(num_test):
        X_train, X_test, y_train, y_test = train_test_split(np.hstack((df_features, preprocessing)), df_target, train_size=2/3)
        rfr = RandomForestRegressor(n_estimators=X_train.shape[1]*2, n_jobs=-1)
        print(f'{i+1:2d}/{num_test:2d}')
        rfr.fit(X_train, y_train)
        mae_erros.append( mean_absolute_error(y_test, rfr.predict(X_test)) )
        medae_erros.append( median_absolute_error(y_test, rfr.predict(X_test)) )
        
        df_features_values[f'model_{i}'] = pd.DataFrame(data=rfr.feature_importances_, index=df_features_values.index.values, columns=[f'model_{i}'])
        
    return df_features_values.T, mae_erros, medae_erros

# Resultados

Gráfico por classificação

In [ ]:
dict_data = {}

for i in range(0, 73):
    print(i)
    dict_data[i] = select_features_rfr(df_fii, i, 10)

0
 1/10
 2/10
 3/10
 4/10
 5/10
 6/10
 7/10
 8/10
 9/10
10/10
1
 1/10
 2/10
 3/10
 4/10
 5/10
 6/10
 7/10
 8/10
 9/10
10/10
2
 1/10
 2/10
 3/10
 4/10
 5/10
 6/10
 7/10
 8/10
 9/10
10/10
3
 1/10
 2/10
 3/10
 4/10
 5/10
 6/10
 7/10
 8/10
 9/10
10/10
4
 1/10
 2/10
 3/10
 4/10
 5/10
 6/10
 7/10
 8/10
 9/10
10/10
5
 1/10
 2/10
 3/10
 4/10
 5/10
 6/10
 7/10
 8/10
 9/10
10/10
6
 1/10
 2/10
 3/10
 4/10
 5/10
 6/10
 7/10
 8/10
 9/10
10/10
7
 1/10
 2/10
 3/10
 4/10
 5/10
 6/10
 7/10
 8/10
 9/10
10/10
8
 1/10
 2/10
 3/10


In [ ]:
x = []
y1 = []
y11 = []
y2 = []
y21 = []

plt.figure(figsize=(10,5))

for i in range(0, 73):
    x.append(i)
    y1.append(np.mean(dict_data[i][1]))
    y11.append(np.std(dict_data[i][1]))
    y2.append(np.mean(dict_data[i][2]))
    y21.append(np.std(dict_data[i][2]))

x = np.asarray(x)
y1 = np.asarray(y1)
y11 = np.asarray(y11)
y2 = np.asarray(y2)
y21 = np.asarray(y21)


plt.plot(x, y1, label='Média')
plt.fill_between(x, y1 - y11, y1 + y11, alpha=0.2, label='Desvio Padrão Média')
plt.plot(x, y2, label='Mediana')
plt.fill_between(x, y2 - y21, y2 + y21, alpha=0.2, label='Desvio Padrão Mediana')
#plt.xticks(x)
plt.xlabel('Menor comprimento da Série')
plt.ylabel('Erro Absoluto')
plt.yscale('log')
plt.legend(loc='best')
plt.show()

In [ ]:
periodo = 36

df_plot = dict_data[periodo][0].mean().groupby(level=[0]).sum().sort_values(ascending=True)*100


plt.figure(figsize=(8,15))
plt.barh(range(df_plot.size), df_plot.values)
plt.yticks(range(df_plot.size), df_plot.index.values)
plt.xlabel('Ordem de grandeza da Importância (%)')
plt.xscale('log')
plt.show()

In [ ]:
periodo = 36

df_plot = dict_data[periodo][0].mean().groupby(level=[0]).sum().sort_values(ascending=True)*100
df_plot = df_plot[df_plot>=1]

plt.figure(figsize=(8, df_plot.size//3))
plt.barh(range(df_plot.size), df_plot.values)
plt.yticks(range(df_plot.size), df_plot.index.values)
plt.xlabel('Ordem de grandeza da Importância (%)')
plt.xscale('log')
plt.show()

In [ ]:
periodo = 36

df_plot = dict_data[periodo][0].mean().groupby(level=[0]).sum().sort_values(ascending=True)*100
df_plot = df_plot[df_plot.sort_values(ascending=True).cumsum() <= 1]

plt.figure(figsize=(8, df_plot.size//3))
plt.barh(range(df_plot.size), df_plot.values)
plt.yticks(range(df_plot.size), df_plot.index.values)
plt.xlabel('Ordem de grandeza da Importância (%)')
plt.xscale('log')
plt.show()

In [ ]:
dataset_filter(df_fii, 36).shape

In [ ]:
from analise_fundos_imobiliarios.run import *
ds = transform_files_cvm_mensal()

In [ ]:
periodo = 36

df_plot = dict_data[periodo][0].mean().groupby(level=[0]).sum().sort_values(ascending=True)*100
df_plot = df_plot[df_plot.sort_values(ascending=True).cumsum() <= 1]

variable = []
porcent = []

for i in df_plot.index.values:
    try:
        porcent.append(ds[i].sum(axis=1)[ds[i].sum(axis=1) != 0].size)
        variable.append(i)
    except:
        pass

df_dis = pd.DataFrame(porcent, columns=['Valor'], index=variable)
(100*df_dis/487).sort_values('Valor', ascending=False).plot.barh(xlabel='Porcentagem de fundo (%)')

In [ ]:
pd.unique(df_dados_fii['Segmento_Atuacao'])

In [ ]:
ds_fii = xr.open_dataset('dataset/cvm/dataset_cvm_metrica_mensal.nc').fillna(0)
df_dados_fii = pd.read_csv('dataset/cvm/dataset_cvm_metrica_mensal_dados.csv', index_col='CNPJ_Fundo')

df_fii = ds_fii.to_array().stack(var=['variable', 'metrica']).to_pandas()
df_fii[['Valor_Ativo']]

In [ ]:
periodo = 36

pd.DataFrame(dict_data[periodo][0].mean()[['Valor_Ativo', 'Total_Passivo']]).groupby('variable').sum()